In [1]:
#working with dataset
import re
import string
import pandas as pd
import numpy as np
import re #regular expression
import json
#preprocessing
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelBinarizer
nltk.download('stopwords')
nltk.download('punkt')
#training the model
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\timber\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\timber\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
#load dataset
df = pd.read_csv('iMDB Dataset.csv')

In [4]:
train_df, test_df = train_test_split(df, test_size=3500) # data split train / test

In [20]:
#data preprocessing
russian_stop_words = stopwords.words("english")
snowball = SnowballStemmer(language="english")
def tokenize_sentence(text):
    text = text.replace(r'\n', ' ')
    pattern =  "[^a-zA-Z0-9]"
    text = re.sub(pattern," ",text) 
    text = text.lower()
    tokens = word_tokenize(text, language="english")
    tokens = [i for i in tokens if i not in string.punctuation]
    tokens = [i for i in tokens if i not in russian_stop_words]
    tokens = [snowball.stem(i) for i in tokens]
    df.sentiment.replace("positive", 1, inplace=True)  
    df.sentiment.replace("negative", 2, inplace=True) 
    return tokens

In [19]:
#train the logistic regression algorithm    
model_pipeline = Pipeline([
    ("vectorizer", TfidfVectorizer(tokenizer = tokenize_sentence, lowercase=False)),
    ("model", LogisticRegression(random_state=0))])

In [21]:
mp = model_pipeline.fit(train_df["review"], train_df["sentiment"])

C:\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [22]:
def predictor(comment):
    return int(model_pipeline.predict([comment])) #returns 1 or 2

In [23]:
# for saving algorithm and preprocessing objects
import dill
filename1 = "mlmodel.pkl"
# save model
dill.dump(mp, open(filename1, "wb"))

In [24]:
predictor('the film is fine')

1

In [25]:
predictor('it was really bad')

2